In [1]:
#Promblem a 
import numpy
from urllib.request import urlopen
import scipy.optimize
import random
from math import exp
from math import log

def parseData(fname):
    for l in urlopen(fname):
        yield eval(l)

In [2]:
print("Reading data...")
data = list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))
print("done")

Reading data...
done


In [9]:
def feature(datum):
    feat = [1, datum['review/taste'], datum['review/appearance'], datum['review/aroma'], datum['review/palate'], datum['review/overall']]
    return feat


random.shuffle(data)
train_data = data[:int(len(data)/3)]
validation_data = data[int(len(data)/3):2*int(len(data)/3)]
test_data = data[2*int(len(data)/3):]
X_train = [feature(d) for d in train_data]
y_train = [d['beer/ABV'] >= 6.5 for d in train_data]
X_validation = [feature(d) for d in validation_data]
y_validation = [d['beer/ABV'] >= 6.5 for d in validation_data]
X_test = [feature(d) for d in test_data]
y_test = [d['beer/ABV'] >= 6.5 for d in test_data]

def inner(x,y):
    return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
    return 1.0 / (1 + exp(-x))

In [10]:
def f(theta, X, y, lam):
    loglikelihood = 0
    for i in range(len(X)):
        logit = inner(X[i], theta)
        loglikelihood -= log(1 + exp(-logit))
        if not y[i]:
            loglikelihood -= logit
    for k in range(len(theta)):
        loglikelihood -= lam * theta[k]*theta[k]
  # for debugging
  # print("ll =" + str(loglikelihood))
    return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
    dl = [0]*len(theta)
    for i in range(len(X)):
        logit = inner(X[i], theta)
        for k in range(len(theta)):
            dl[k] += X[i][k] * (1 - sigmoid(logit))
            if not y[i]:
                dl[k] -= X[i][k]
    for k in range(len(theta)):
        dl[k] -= lam*2*theta[k]
    return numpy.array([-x for x in dl])



In [11]:

def train(lam):
    theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X_train[0]), fprime, pgtol = 10, args = (X_train, y_train, lam))
    return theta


In [12]:
def performance(theta, X, y):
    scores = [inner(theta,x) for x in X]
    predictions = [s > 0 for s in scores]
    correct = [(a==b) for (a,b) in zip(predictions,y)]
    acc = sum(correct) * 1.0 / len(correct)
    return acc

In [13]:
#question 1
lam = 1.0

theta = train(lam)
acc = performance(theta,X_train,y_train)
print("train lambda = " + str(lam) + ":\taccuracy=" + str(acc))


acc = performance(theta,X_validation,y_validation)
print("validation lambda = " + str(lam) + ":\taccuracy=" + str(acc))

acc = performance(theta,X_test,y_test)
print("test lambda = " + str(lam) + ":\taccuracy=" + str(acc))

train lambda = 1.0:	accuracy=0.717508700348014
validation lambda = 1.0:	accuracy=0.718408736349454
test lambda = 1.0:	accuracy=0.7215022798176146


In [14]:
#question 2
scores = [inner(theta,x) for x in X_test]
predictions = [s > 0 for s in scores]
correct = [(a==b) for (a,b) in zip(predictions,y_test)]
print('positive',sum(predictions))
print('negetive',len(predictions)-sum(predictions))
True_positive =0
True_negetive =0
False_positive = 0
False_negetive = 0
i=0
for i in range(len(predictions)) :
    if predictions[i] == True:
        if y_test[i] == True:
            True_positive =True_positive +1
            i=i+1
        else:
            False_positive =False_positive +1
            i=i+1
    elif y_test[i] == True:
        True_negetive = True_negetive + 1
        i=i+1
    else:
        False_negetive= False_negetive+1
        i=i+1
print('True_positive= ',True_positive,'True_negetive =',True_negetive,'False_positive = ',False_positive,'False_negetive = ',False_negetive)

positive 12340
negetive 4328
True_positive=  9105 True_negetive = 1407 False_positive =  3235 False_negetive =  2921


In [15]:
#question 3 
#To assign greater importance to False Positives compared to False Negatives,I need to change the def f.I will add two more judgements and make it into four situation TP,TN,NP,NN.
#Then I apply 10 times weight for the FP situations(if not y[i] and x[i]>0 )
#Finally,add two more judgements and change the coefficient of FP.

In [16]:
#question 4
import numpy as np
a= [0,0.01,0.1,1,100]
array1=[]
for lam in a:
    theta = train(lam)
    acc = performance(theta,X_validation,y_validation)
    array1.append(acc)
    print("validation lambda = " + str(lam) + ":\taccuracy=" + str(acc))

validation lambda = 0:	accuracy=0.718948757950318
validation lambda = 0.01:	accuracy=0.719188767550702
validation lambda = 0.1:	accuracy=0.718648745949838
validation lambda = 1:	accuracy=0.718408736349454
validation lambda = 100:	accuracy=0.6694467778711148


In [17]:
print('The best lambda is ',a[array1.index(max(array1))])
lam = a[array1.index(max(array1))]

theta = train(lam)
acc = performance(theta,X_train,y_train)
print("train lambda = " + str(lam) + ":\taccuracy=" + str(acc))


acc = performance(theta,X_validation,y_validation)
print("validation lambda = " + str(lam) + ":\taccuracy=" + str(acc))

acc = performance(theta,X_test,y_test)
print("test lambda = " + str(lam) + ":\taccuracy=" + str(acc))

The best lambda is  0.01
train lambda = 0.01:	accuracy=0.718288731549262
validation lambda = 0.01:	accuracy=0.719188767550702
test lambda = 0.01:	accuracy=0.7224022078233742
